In [5]:
import numcodecs
import zarr

In [1]:
import h5py

In [1]:
import cosilico_py.preprocessing.platform_helpers.x10 as x10

In [1]:
from rich import print

In [2]:
from ome_types import from_xml
import tifffile
import dask.array as da

In [19]:
image

ZarrTiffStore

In [20]:
ome_model

OME(
   plates=[{'id': 'Plate:1', 'well_origin_x': -0.0, 'well_origin_x_unit': <UnitsLength.MICROMETER: 'µm'>, 'well_origin_y': -0.0, 'well_origin_y_unit': <UnitsLength.MICROMETER: 'µm'>}],
   instruments=[{'microscope': {'manufacturer': '10x Genomics', 'model': 'Xenium', 'kind': 'microscope'}, 'id': 'Instrument:1'}],
   images=[<1 field_type>],
   uuid='urn:uuid:6f6e3c8f-db12-11ed-a5c7-0644c1d5d2e3',
   creator='tifffile.py 2021.4.8',
)

In [63]:
# ome_tiff_path = '/Users/erikstorrs/Downloads/Xenium_V1_FFPE_Human_Brain_Healthy_With_Addon_outs/morphology_focus.ome.tif'
ome_tiff_path = '/Users/erikstorrs/Downloads/Xenium_V1_FFPE_Human_Brain_Healthy_With_Addon_outs/morphology.ome.tif'
# ome_tiff_path = '/Users/erikstorrs/Downloads/Xenium_Prime_Breast_Cancer_FFPE_outs/morphology_focus/morphology_focus_0002.ome.tif'
# ome_tiff_path = '/Users/erikstorrs/Downloads/Xenium_V1_Human_Ductal_Adenocarcinoma_FFPE_outs/morphology_focus/morphology_focus_0000.ome.tif'
ome_metadata = tifffile.TiffFile(ome_tiff_path).ome_metadata
ome_model = from_xml(ome_metadata)

with tifffile.TiffFile(ome_tiff_path) as tif:
#     z = tif.aszarr()
#     zz = tif.series[0].aszarr()
    z = tif.series[0].pages[0].aszarr()
    image = da.from_zarr(z)
#     image = da.from_zarr(zz)
#     image = da.from_zarr(image, chunks=(1, 2048, 2048))

In [64]:
ome_model.images[0]

Image(
   id='Image:0',
   name='Image0',
   instrument_ref={'id': 'Instrument:1'},
   pixels={'channels': [{'id': 'Channel:0:0', 'name': 'DAPI', 'samples_per_pixel': 1}], 'tiff_data_blocks': [{'plane_count': 13}], 'id': 'Pixels:0', 'dimension_order': <Pixels_DimensionOrder.XYZCT: 'XYZCT'>, 'type': <PixelType.UINT16: 'uint16'>, 'size_x': 36955, 'size_y': 27282, 'size_z': 13, 'size_c': 1, 'size_t': 1, 'physical_size_x': 0.2125, 'physical_size_y': 0.2125, 'physical_size_z': 3.0},
)

In [65]:
for s in tif.series:
    print(len(s.pages), s)
    for p in s.pages:
        print(p)

13 TiffPageSeries 0  'Image0'  13x27282x36955  uint16  ZYX  ome  8 Levels  13 Pages
TiffPage 0 @16  27282x36955  uint16  minisblack tiled jpeg2000  ome
TiffFrame 1 @13939344  27282x36955  uint16
TiffFrame 2 @146243052  27282x36955  uint16
TiffFrame 3 @394109272  27282x36955  uint16
TiffFrame 4 @697471996  27282x36955  uint16
TiffFrame 5 @998401786  27282x36955  uint16
TiffFrame 6 @1298411118  27282x36955  uint16
TiffFrame 7 @1597568094  27282x36955  uint16
TiffFrame 8 @1896267556  27282x36955  uint16
TiffFrame 9 @2195048900  27282x36955  uint16
TiffFrame 10 @2496114066  27282x36955  uint16
TiffFrame 11 @2786268448  27282x36955  uint16
TiffFrame 12 @2959810478  27282x36955  uint16


In [49]:
for p in tif.pages:
    print(p)

TiffPage 0 @16  74945x51265  uint16  minisblack tiled jpeg2000  ome


In [59]:
image

dask.array<from-zarr, shape=(27282, 36955), dtype=uint16, chunksize=(1024, 1024), chunktype=numpy.ndarray>

In [43]:
s = tif.series[0]
s

<tifffile.TiffPageSeries 0 ome>

In [46]:
s.aszarr()

AssertionError: 

In [6]:
import pandas as pd

In [8]:
df = pd.read_csv('/Users/erikstorrs/Downloads/Xenium_Prime_Breast_Cancer_FFPE_outs/metrics_summary.csv')
df

,run_name,cassette_name,region_name,panel_name,panel_design_id,predesigned_panel_id,region_area,total_cell_area,total_high_quality_decoded_transcripts,fraction_transcripts_decoded_q20,...,fraction_of_ambiguous_cell_mask_pixels,fraction_of_nucleus_polygons_removed,fraction_of_cell_polygons_removed,fraction_of_nuclei_without_cell,number_of_cell_non_simple_polygons,number_of_cell_multi_polygons,number_of_nucleus_non_simple_polygons,number_of_nucleus_multi_polygons,segmented_cell_imported_frac,segmented_cell_imported_count
0,Xenium Prime + 100g Custom,NaN,Breast_Cancer,Xenium Human 5K with Cell Typing + HPV-16/18 a...,NaN,hAtlas_v1.1,1.551598e+08,4.553694e+07,80158637,0.86385,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df.to_json()

'{"run_name":{"0":"Xenium Prime + 100g Custom"},"cassette_name":{"0":null},"region_name":{"0":"Breast_Cancer"},"panel_name":{"0":"Xenium Human 5K with Cell Typing + HPV-16\\/18 and SNV Add-on"},"panel_design_id":{"0":null},"predesigned_panel_id":{"0":"hAtlas_v1.1"},"region_area":{"0":155159780.8498440087},"total_cell_area":{"0":45536938.5737275034},"total_high_quality_decoded_transcripts":{"0":80158637},"fraction_transcripts_decoded_q20":{"0":0.8638496091},"fraction_predesigned_transcripts_decoded_q20":{"0":0.8662174711},"fraction_custom_transcripts_decoded_q20":{"0":0.8272998337},"nuclear_transcripts_per_100um2":{"0":196.2993157169},"decoded_transcripts_per_100um2":{"0":157.1654402812},"adjusted_negative_control_probe_rate":{"0":0.0013769231},"adjusted_negative_control_codeword_rate":{"0":0.0002238943},"adjusted_genomic_control_probe_rate":{"0":0.0336418666},"negative_control_probe_counts_per_control_per_cell":{"0":0.00002918},"genomic_control_probe_counts_per_control_per_cell":{"0":0

In [11]:
from dateutil import parser

dt = parser.isoparse("2024-06-13T21:22:41Z")
dt

datetime.datetime(2024, 6, 13, 21, 22, 41, tzinfo=tzutc())

In [2]:
print('[bold red]Sign in attempt failed.[/bold red]')

Sign in attempt failed.

In [5]:
print('[green]Sign in successfull.[/green]')

Sign in successfull.

In [2]:
adata = x10.read_10x_h5('/Users/erikstorrs/Downloads/Xenium_Prime_Breast_Cancer_FFPE_outs/cell_feature_matrix.h5')
adata.obs

""
aaaaaohf-1
aaaabkoj-1
aaaafefl-1
aaaahfjm-1
aaaahjao-1
...
oimkalkm-1
oimkankb-1
oimkapok-1
oimkbcmb-1


In [9]:
import pandas as pd
s = pd.Series(pd.Categorical(['a', 'a', 'b', 'd']))
s.cat.categories.to_list()

['a', 'b', 'd']

In [2]:
import pandas as pd
dtype_dict = {
        "feature_name": "category",
        "is_gene": "bool",
        "codeword_category": "category",
        "x_location": "float32",
        "y_location": "float32",
        "transcript_id": "uint64",
        "qv": "float32",
    }
source = pd.read_parquet(
    "/Users/erikstorrs/Downloads/Xenium_Prime_Breast_Cancer_FFPE_outs/transcripts.parquet",
    columns=["feature_name", "is_gene", "codeword_category", "x_location", "y_location", "transcript_id", "qv"],  # Read only needed columns
    engine="pyarrow"  # Ensure efficient Parquet reading
).astype(dtype_dict)  # Convert data types immediately

In [3]:
adata.var

,gene_ids,gene_names
0,ENSG00000166535,A2ML1
1,ENSG00000127837,AAMP
2,ENSG00000131043,AAR2
3,ENSG00000266967,AARSD1
4,ENSG00000183044,ABAT
...,...,...
9470,DeprecatedCodeword_18632,DeprecatedCodeword_18632
9471,DeprecatedCodeword_18637,DeprecatedCodeword_18637
9472,DeprecatedCodeword_18639,DeprecatedCodeword_18639
9473,DeprecatedCodeword_18646,DeprecatedCodeword_18646


In [4]:
adata.X.shape

(699110, 9475)

In [98]:
output_filepath = '/Users/erikstorrs/Downloads/tempsdf.zarr.zip'
store = zarr.storage.ZipStore(output_filepath, mode='w')
root = zarr.group(store=store, overwrite=True)
g = root.create_group("g1")
x = np.asarray(['hello', 'aaaaaaaaddddddddeeeeeeeeeeaaaaaaaaddddddddeeeeeeeeeeaaaaaaaaddddddddeeeeeeeeeeaaaaaaaaddddddddeeeeeeeeeeaaaaaaaaddddddddeeeeeeeeee']).astype(str)
x = np.asarray(['a', 'b'], dtype=str)
z = g.create_array("testd", shape=(len(x),), chunks=(len(x),), dtype='U50')
z[:] = x


In [99]:
z[:]

array(['a', 'b'], dtype=StringDType())

In [100]:
np.asarray(np.asarray([1,2]))

array([1, 2])

In [95]:
g = root.create_group("g1")

ContainsGroupError: A group exists in store ZipStore('zip:///Users/erikstorrs/Downloads/tempsdf.zarr.zip') at path 'g1'.

In [78]:
# feature_meta_group.create_dataset("feature_names", shape=(len(names),), chunks=(len(names),), dtype=object, object_codec=numcodecs.VLenUTF8())
x = ['hello', 'worasdfsadfasdfasdfasdfasdfasdfasdfasdfasdfasdfasdfasdfasdfasdfasdfasdfasdfadslddk']
g.create_array("testd", shape=(len(x),), chunks=(len(x),), dtype='U50')



<Array zip:///Users/erikstorrs/Downloads/tempsdf.zarr.zip/g1/testd shape=(2,) dtype=StringDType()>

In [79]:
g[:] = np.asarray(x).astype(str)

AssertionError: 

In [27]:
store.close()

In [59]:
import numpy as np

In [81]:
import zarr
# store = zarr.storage.MemoryStore()
# store = '/Users/erikstorrs/Downloads/alsdkfj.zarr.zip'
store = zarr.storage.ZipStore('/Users/erikstorrs/Downloads/hghgh.zarr.zip', mode='w')
root = zarr.group(store=store, overwrite=True)
g = root.create_group('hello')
z = g.create_array(name='test', shape=(10, 10), chunks=(10, 10), dtype='U50')
# z = zarr.create_array(store=store, shape=(10000, 10000), chunks=(1000, 1000), dtype='int32')
z

<Array zip:///Users/erikstorrs/Downloads/hghgh.zarr.zip/hello/test shape=(10, 10) dtype=StringDType()>

In [82]:
z[:] = np.full((10, 10), 1, dtype=np.int32).astype(str)

In [83]:
store.close()